# **Ejemplo Rápido de Evaluación**

Demostración práctica del sistema de evaluación comprehensiva.

Este notebook te muestra cómo:
1. Cargar diferentes tipos de benchmarks
2. Evaluar múltiples heurísticas
3. Analizar y comparar resultados

In [ ]:
%run librerias.ipynb
%run benchmark_loader.ipynb
%run heuristics.ipynb

## **EJEMPLO 1: Grafo Simple Individual**

In [ ]:
print("="*60)
print("EJEMPLO 1: Evaluando Grafo de Petersen")
print("="*60)

G, nombre = generar_grafo_simple('petersen')
stats = estadisticas_grafo(G, nombre)

print(f"\nGrafo: {stats['nombre']}")
print(f"Nodos: {stats['nodos']}, Aristas: {stats['aristas']}")
print(f"Densidad: {stats['densidad']:.4f}")
print(f"Grado máximo: {stats['grado_max']}")
print(f"Número cromático teórico: {stats['chi_teorico']}")
print(f"Cota superior (Δ+1): {stats['cota_superior']}")

print("\nProbando diferentes ordenamientos:")

order_random = ordenamiento_aleatorio(G)
coloring_random = greedy_coloring(G, order_random)
metrics_random = evaluar_coloracion(G, coloring_random)
print(f"  Random: {metrics_random['num_colores']} colores")

order_degree = ordenamiento_grado_desc(G)
coloring_degree = greedy_coloring(G, order_degree)
metrics_degree = evaluar_coloracion(G, coloring_degree)
print(f"  Largest Degree First: {metrics_degree['num_colores']} colores")

order_wp = ordenamiento_welsh_powell(G)
coloring_wp = greedy_coloring(G, order_wp)
metrics_wp = evaluar_coloracion(G, coloring_wp)
print(f"  Welsh-Powell: {metrics_wp['num_colores']} colores")

coloring_dsatur = dsatur_coloring(G)
metrics_dsatur = evaluar_coloracion(G, coloring_dsatur)
print(f"  DSATUR: {metrics_dsatur['num_colores']} colores")

print(f"\n✓ Mejor resultado: {min(metrics_random['num_colores'], metrics_degree['num_colores'], metrics_wp['num_colores'], metrics_dsatur['num_colores'])} colores")

## **EJEMPLO 2: Comparación en Múltiples Grafos Simples**

In [ ]:
print("\n" + "="*60)
print("EJEMPLO 2: Comparación en Suite Simple")
print("="*60)

grafos_prueba = [
    generar_grafo_simple('petersen'),
    generar_grafo_simple('cycle', n=10),
    generar_grafo_simple('wheel', n=10),
    generar_grafo_simple('complete', n=5),
]

resultados_comparacion = []

for G, nombre in grafos_prueba:
    print(f"\nEvaluando: {nombre}")
    
    order_degree = ordenamiento_grado_desc(G)
    coloring_degree = greedy_coloring(G, order_degree)
    metrics_degree = evaluar_coloracion(G, coloring_degree)
    
    coloring_dsatur = dsatur_coloring(G)
    metrics_dsatur = evaluar_coloracion(G, coloring_dsatur)
    
    resultados_comparacion.append({
        'Grafo': nombre,
        'Nodos': G.number_of_nodes(),
        'Aristas': G.number_of_edges(),
        'LDF': metrics_degree['num_colores'],
        'DSATUR': metrics_dsatur['num_colores'],
        'Mejor': min(metrics_degree['num_colores'], metrics_dsatur['num_colores'])
    })
    
    print(f"  LDF: {metrics_degree['num_colores']} colores")
    print(f"  DSATUR: {metrics_dsatur['num_colores']} colores")

df_comparacion = pd.DataFrame(resultados_comparacion)
print("\n" + "="*60)
print("TABLA COMPARATIVA")
print("="*60)
print(df_comparacion.to_string(index=False))

## **EJEMPLO 3: Grafo Complejo (Scale-Free)**

In [ ]:
print("\n" + "="*60)
print("EJEMPLO 3: Red Scale-Free (Barabási-Albert)")
print("="*60)

G, nombre = generar_benchmark_complejo('scale_free', n=100, m=3)
stats = estadisticas_grafo(G, nombre)

print(f"\nGrafo: {stats['nombre']}")
print(f"Nodos: {stats['nodos']}, Aristas: {stats['aristas']}")
print(f"Densidad: {stats['densidad']:.4f}")
print(f"Grado promedio: {stats['grado_promedio']:.2f}")
print(f"Grado máximo: {stats['grado_max']}")
print(f"Clustering: {stats['clustering']:.4f}")

print("\nEvaluando heurísticas (puede tomar unos segundos)...")

import time

t0 = time.time()
order_degree = ordenamiento_grado_desc(G)
coloring_degree = greedy_coloring(G, order_degree)
metrics_degree = evaluar_coloracion(G, coloring_degree)
t1 = time.time()
print(f"  LDF: {metrics_degree['num_colores']} colores ({t1-t0:.4f}s)")

t0 = time.time()
order_wp = ordenamiento_welsh_powell(G)
coloring_wp = greedy_coloring(G, order_wp)
metrics_wp = evaluar_coloracion(G, coloring_wp)
t1 = time.time()
print(f"  Welsh-Powell: {metrics_wp['num_colores']} colores ({t1-t0:.4f}s)")

t0 = time.time()
coloring_dsatur = dsatur_coloring(G)
metrics_dsatur = evaluar_coloracion(G, coloring_dsatur)
t1 = time.time()
print(f"  DSATUR: {metrics_dsatur['num_colores']} colores ({t1-t0:.4f}s)")

print(f"\n✓ DSATUR típicamente da los mejores resultados en redes scale-free")

## **EJEMPLO 4: Análisis de Escalabilidad**

In [ ]:
print("\n" + "="*60)
print("EJEMPLO 4: Escalabilidad con Tamaño del Grafo")
print("="*60)

tamaños = [20, 50, 100, 200]
resultados_escalabilidad = []

for n in tamaños:
    print(f"\nGenerando grafo random con {n} nodos...")
    G, nombre = generar_grafo_simple('random', n=n, p=0.1)
    
    t0 = time.time()
    order_degree = ordenamiento_grado_desc(G)
    coloring_degree = greedy_coloring(G, order_degree)
    metrics_degree = evaluar_coloracion(G, coloring_degree)
    tiempo_ldf = time.time() - t0
    
    t0 = time.time()
    coloring_dsatur = dsatur_coloring(G)
    metrics_dsatur = evaluar_coloracion(G, coloring_dsatur)
    tiempo_dsatur = time.time() - t0
    
    resultados_escalabilidad.append({
        'Nodos': n,
        'Aristas': G.number_of_edges(),
        'LDF_colores': metrics_degree['num_colores'],
        'LDF_tiempo': tiempo_ldf,
        'DSATUR_colores': metrics_dsatur['num_colores'],
        'DSATUR_tiempo': tiempo_dsatur
    })
    
    print(f"  LDF: {metrics_degree['num_colores']} colores en {tiempo_ldf:.4f}s")
    print(f"  DSATUR: {metrics_dsatur['num_colores']} colores en {tiempo_dsatur:.4f}s")

df_escalabilidad = pd.DataFrame(resultados_escalabilidad)
print("\n" + "="*60)
print("ANÁLISIS DE ESCALABILIDAD")
print("="*60)
print(df_escalabilidad.to_string(index=False))

print("\n💡 Observación: DSATUR da mejores resultados pero es más lento en grafos grandes")

## **EJEMPLO 5: Cargando Archivo Externo (DIMACS/SNAP)**

In [ ]:
print("\n" + "="*60)
print("EJEMPLO 5: Carga de Archivos Externos")
print("="*60)

print("\nPara cargar un archivo DIMACS:")
print("  benchmarks = cargar_benchmark('dimacs', filepath='datasets/queen5_5.col')")

print("\nPara cargar un archivo SNAP:")
print("  benchmarks = cargar_benchmark('snap', filepath='datasets/facebook.txt')")

print("\nEjemplo con archivo (si existe):")
print("  G, nombre = benchmarks[0]")
print("  resultados = evaluar_todas_heuristicas(G, model=None, repeticiones=3)")

print("\n📚 Descarga benchmarks DIMACS de:")
print("  https://mat.tepper.cmu.edu/COLOR/instances.html")

print("\n📚 Descarga datasets SNAP de:")
print("  http://snap.stanford.edu/data/")

## **RESUMEN Y PRÓXIMOS PASOS**

In [ ]:
print("\n" + "#"*60)
print("# RESUMEN DE EJEMPLOS")
print("#"*60)

print("\n✅ Has visto cómo:")
print("  1. Evaluar un grafo individual")
print("  2. Comparar múltiples grafos")
print("  3. Trabajar con grafos complejos")
print("  4. Analizar escalabilidad")
print("  5. Cargar archivos externos")

print("\n🚀 PRÓXIMOS PASOS:")
print("  1. Ejecuta 'evaluation_comprehensive.ipynb' para evaluación completa")
print("  2. Lee 'GUIA_EVALUACION.md' para metodología detallada")
print("  3. Descarga benchmarks DIMACS/SNAP para comparar con literatura")
print("  4. Entrena tu modelo GNN y evalúalo con el sistema")

print("\n📖 DOCUMENTACIÓN:")
print("  • benchmark_loader.ipynb - Carga de datasets")
print("  • evaluation_comprehensive.ipynb - Evaluación completa")
print("  • GUIA_EVALUACION.md - Guía detallada")
print("  • ejemplo_evaluacion_rapida.ipynb - Este notebook")

print("\n" + "#"*60)